### TODO Recording

- Here we will set up a tracking server and log our model's experiments, runs, etc to a Sqlite DB
- Go to this URL

`https://www.sqlite.org/index.html`

- Click on the Download button
- Choose the option for Precompiled Binaries for Mac OS X (x86)
- Download the ZIP file
- Move the Zip to /Users/Loonycorn and unzip - this will create a subfolder sqlite-tools-osx-x86-3420000

#### Terminal (new window)
- Open up a terminal and go to the home directory by typing

`cd`

- Open up the zshrc file to add an environment variable (this file will be different for different shells bash, cshrc, zshrc etc)

`nano .zshrc`

- Add this line

`export PATH="/Users/vitthalsrinivasan/sqlite-tools-osx-x86-3420000:$PATH"`

- Ctrl + X yes on save

- Run this command

`source .zshrc`

#### SQLite database

- Go to ~/projects/mlflow

`cd ~/projects/mlflow`

`mkdir database`

`cd database`

- Run the command to create a SQLite db file (copy NUL mlflow.db on Windows)

`touch mlflow.db`

- Log into the SQLite DB

`sqlite3 mlflow.db`

- Run the command to show tables (there should be nothing)

`.tables`

#### Run the tracking server

- Go to the terminal tab where you have the MLflow UI running and kill it
-Checking mlflow version by running mlflow --version which is 2.3.2
- We need to upgrade mlflow to latest version 2.4.1
-run pip install mlflow -U
-Verifying the upgraded version  by running mlflow --version

- Now run the tracking server

`
mlflow server \
    --backend-store-uri sqlite:////Users/loonycorn/projects/mlflow/database/mlflow.db \
    --host localhost
     -p 5000
`

- On the web browser go to http://localhost:5000 (you should see the familiar UI)

- Once the server is running go back to the tab where you are logged into SQLite

- Run

`.tables`

- Many tables have been created for the different metadata that will be logged

- Run these SQL commands (only experiments table will have something)

`select * from experiments;`

`.schema runs`

`select * from runs;`

`select * from metrics;`

- Now you can continue on this notebook

#### Notes:

- The windows installation of SQLite is exactly the same as the MacOS installation (https://www.tutorialspoint.com/sqlite/sqlite_installation.htm)
- We need to unzip the file - place it in a known directory and configure the PATH env variable to point to that directory
- Add a subtitle to this link https://www.computerhope.com/issues/ch000549.htm which shows how you can configure the PATH environment variable for different versions of Windows

Installing statsmodels package

In [1]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [39]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [40]:
ecom_data = pd.read_csv('datasets/Ecommerce_data.csv')

ecom_data.head()

\tEmail  \
0      mstephenson@fernandez.com   
1              hduke@hotmail.com   
2               pallen@yahoo.com   
3        riverarebecca@gmail.com   
4  mstephens@davidson-herman.com   

                                             Address            Avatar  \
0         835 Frank TunnelWrightmouth, MI 82180-9605            Violet   
1       4547 Archer CommonDiazchester, CA 06566-8576         DarkGreen   
2  24645 Valerie Unions Suite 582Cobbborough, DC ...            Bisque   
3     1414 David ThroughwayPort Jason, OH 22070-1220       SaddleBrown   
4  14023 Rodriguez PassagePort Jacobville, PR 372...  MediumAquaMarine   

   Time on App  Time on Website  Length of Membership  Yearly Amount Spent  
0        12.66            39.58                  4.08               587.95  
1        11.11            37.27                  2.66               392.20  
2        11.33            37.11                  4.10               487.55  
3        13.72            36.72                  3.12               581.85  
4        12.80            37.54                  4.45               599.41

In [41]:
ecom_data = ecom_data.drop(columns = ['\tEmail', 'Address', 'Avatar'])
    
numeric_transformer = Pipeline(steps = [('scaler', StandardScaler())])
   
X = ecom_data.drop(columns = ['Yearly Amount Spent'])

y = ecom_data['Yearly Amount Spent']

In [42]:
import mlflow

mlflow.set_tracking_uri('http://localhost:5000')

In [43]:
mlflow.get_tracking_uri()

'http://localhost:5000'

In [44]:
experiment_id = mlflow.create_experiment('ecommerce_revenue_prediction')

mlflow.set_experiment('ecommerce_revenue_prediction')

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1687330840008, experiment_id='1', last_update_time=1687330840008, lifecycle_stage='active', name='ecommerce_revenue_prediction1', tags={}>

### TODO Recording

- Go to the MLflow UI and show the experiment
- Go to the terminal where you are logged into SQLite3

- Run

`select * from experiments;`

- One more experiment should be here (our newly created one)

In [45]:
import mlflow.sklearn

mlflow.sklearn.autolog()

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

with mlflow.start_run(run_name = 'regression using sklearn') as run1:          
    reg_model = LinearRegression()

    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size = 0.2, random_state = 123)
    
    reg_model.fit(X_train, y_train)
    
    y_pred =  reg_model.predict(X_test)
    
    mean_abs_error = mean_absolute_error(y_test, y_pred)
    root_mean_sq_error = mean_squared_error(y_test, y_pred, squared = False)
    r_sq_score = r2_score(y_test, y_pred)
    
    mlflow.log_metrics(dict(zip(X_train.columns, reg_model.coef_)))
    
    mlflow.set_tag('Regressor', 'linear regression sklearn')

### TODO Recording:

- Go to the MLflow UI and show the run
- Expand the sections and show the details (especially metrics)

In [46]:
import mlflow.statsmodels

mlflow.statsmodels.autolog()

import statsmodels.api as sm
from mlflow.models.signature import infer_signature

with mlflow.start_run(run_name = 'regression using statsmodels') as run2:   
    
    X_train = sm.add_constant(X_train)
    X_test = sm.add_constant(X_test)
    
    model = sm.OLS(y_train, X_train).fit()
    y_pred =  model.predict(X_test)
    
    mlflow.log_metrics(model.params)
    
    # Even with autologging statsmodels does not log the signature
    signature = infer_signature(X_train, model.predict(X_train))
    
    mlflow.sklearn.log_model(model, 'MLmodel', signature = signature)

    mlflow.set_tag('Regressor', 'linear regression statsmodels')

### TODO Recording:

- Go to the MLflow UI and show the run
- Expand the sections and show the details (especially metrics)

- Go to the terminal where you are logged into sqlite

- Run

`select * from runs;`

- There should be 2 runs in here

In [47]:
run_id = run2.info.run_id

run_id

'b04e3f8df5b5415cb92706fff23e9d44'

In [48]:
import mlflow
logged_model = f'runs:/{run_id}/model'

loaded_model = mlflow.pyfunc.load_model(logged_model)

preds = loaded_model.predict(X_test)

preds

2023/06/21 12:34:52 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


229    427.923462
337    479.293135
327    614.900100
416    498.072876
306    418.217534
          ...    
252    421.712909
102    416.946464
261    532.341771
351    554.163370
461    504.989919
Length: 100, dtype: float64

In [49]:
actual_predictions_df = pd.DataFrame({'actual': y_test, 'predicted': preds})

actual_predictions_df.sample(15)

actual   predicted
306  387.53  418.217534
55   483.67  473.103811
392  549.13  549.528292
171  439.89  478.317163
41   501.87  515.349817
440  499.14  497.271265
237  376.34  414.045310
204  568.72  535.502980
431  475.07  449.033740
317  404.82  416.516836
15   461.78  510.938435
320  368.65  387.166425
75   478.72  480.074324
93   439.07  423.600018
49   416.36  453.631510

In [50]:
model_name = 'revenue_prediction_model'
 
model_version = mlflow.register_model(f'runs:/{run_id}/model', model_name)

Successfully registered model 'revenue_prediction_model'.
2023/06/21 12:34:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: revenue_prediction_model, version 1
Created version '1' of model 'revenue_prediction_model'.


### TODO Recording:

- Go to the MLFlow UI and click on the Models tab and show the registered model
- Change the stage of the model to Production

In [52]:
stage = 'Production'
 
loaded_model = mlflow.pyfunc.load_model(
    model_uri = f'models:/{model_name}/{stage}'
)
 
print(f'R2 score: {r2_score(y_test, loaded_model.predict(X_test))}')

2023/06/21 12:37:36 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


R2 score: 0.9030727941952026


### TODO Recording

- Go to the terminal where you are logged into sqlite

- Run

`select * from model_versions;`

- There should be one version there

Again uninstalling 2.4.1 and installing mlflow version 2.3.2